In [1]:
from keras.datasets import mnist
from keras.layers import Dense,Flatten
from keras.layers import Conv2D,MaxPooling2D
from keras.models import Sequential

In [2]:
(x_train,y_train),(x_test,y_test)=mnist.load_data()
x_train.shape

11490434/11490434 [==============================] - 2s 0us/step


(60000, 28, 28)

In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.image import resize

In [4]:
# Normalize the images
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Convert labels to one-hot encoding
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

def image_generator(images, labels, batch_size):
    while True:
        for start in range(0, len(images), batch_size):
            end = min(start + batch_size, len(images))
            batch_images = images[start:end]
            batch_labels = labels[start:end]
            # Resize images and expand to 3 channels
            batch_images_resized = np.array([resize(image.reshape(28, 28, 1), [224, 224]).numpy() for image in batch_images])
            batch_images_resized = np.repeat(batch_images_resized, 3, axis=-1)
            yield batch_images_resized, batch_labels

# Define batch size
batch_size = 32

# Create generators
train_gen = image_generator(x_train, y_train, batch_size)
test_gen = image_generator(x_test, y_test, batch_size)

In [5]:
x_batch, y_batch = next(train_gen)

print(f"Batch images shape: {x_batch.shape}")
print(f"Batch labels shape: {y_batch.shape}")

Batch images shape: (32, 224, 224, 3)
Batch labels shape: (32, 10)


In [6]:
import tensorflow as tf
from tensorflow.keras import layers, Model, Input
from tensorflow.keras.layers import Conv2D, DepthwiseConv2D, BatchNormalization, ReLU, GlobalAveragePooling2D, Dense

def depth_block(x, strides):
    x = DepthwiseConv2D(3, strides=strides, padding='same', use_bias=False)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    return x

def single_conv_block(x, filters):
    x = Conv2D(filters, 1, use_bias=False)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    return x

def combo_layer(x, filters, strides):
    x = depth_block(x, strides)
    x = single_conv_block(x, filters)
    return x

def MobileNet(input_shape=(224, 224, 3), n_classes=10):
    input = Input(input_shape)
    x = Conv2D(32, 3, strides=(2, 2), padding='same', use_bias=False)(input)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = combo_layer(x, 64, strides=(1, 1))
    x = combo_layer(x, 128, strides=(2, 2))
    x = combo_layer(x, 128, strides=(1, 1))
    x = combo_layer(x, 256, strides=(2, 2))
    x = combo_layer(x, 256, strides=(1, 1))
    x = combo_layer(x, 512, strides=(2, 2))
    for _ in range(5):
        x = combo_layer(x, 512, strides=(1, 1))
    x = combo_layer(x, 1024, strides=(2, 2))
    x = combo_layer(x, 1024, strides=(1, 1))
    x = GlobalAveragePooling2D()(x)
    output = Dense(n_classes, activation='softmax')(x)
    model = Model(input, output)
    return model

# Specify the input shape and number of classes
input_shape = (224, 224, 3)
n_classes = 10

# Build the model
model = MobileNet(input_shape, n_classes)

# Print the model summary
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 112, 112, 32)      864       
                                                                 
 batch_normalization (Batch  (None, 112, 112, 32)      128       
 Normalization)                                                  
                                                                 
 re_lu (ReLU)                (None, 112, 112, 32)      0         
                                                                 
 depthwise_conv2d (Depthwis  (None, 112, 112, 32)      288       
 eConv2D)                                                        
                                                                 
 batch_normalization_1 (Bat  (None, 112, 112, 32)      128   

In [8]:
import keras

In [9]:
model.compile(loss=keras.metrics.categorical_crossentropy,optimizer=keras.optimizers.Adam(),metrics=['accuracy'])

In [10]:
model.fit(
    train_gen,
    steps_per_epoch=len(x_train) // batch_size,
    epochs=10,
    validation_data=test_gen,
    validation_steps=len(x_test) // batch_size
)

Epoch 1/10
1875/1875 [==============================] - 342s 172ms/step - loss: 0.1276 - accuracy: 0.9596 - val_loss: 0.1008 - val_accuracy: 0.9710
Epoch 2/10
1875/1875 [==============================] - 323s 172ms/step - loss: 0.0497 - accuracy: 0.9845 - val_loss: 0.0481 - val_accuracy: 0.9849
Epoch 3/10
1875/1875 [==============================] - 307s 164ms/step - loss: 0.0401 - accuracy: 0.9875 - val_loss: 0.0477 - val_accuracy: 0.9870
Epoch 4/10
1875/1875 [==============================] - 316s 169ms/step - loss: 0.0324 - accuracy: 0.9896 - val_loss: 0.0294 - val_accuracy: 0.9904
Epoch 5/10
1875/1875 [==============================] - 316s 169ms/step - loss: 0.0260 - accuracy: 0.9918 - val_loss: 0.0262 - val_accuracy: 0.9922
Epoch 6/10
1875/1875 [==============================] - 299s 160ms/step - loss: 0.0227 - accuracy: 0.9928 - val_loss: 0.0437 - val_accuracy: 0.9855
Epoch 7/10
1875/1875 [==============================] - 308s 165ms/step - loss: 0.0190 - accuracy: 0.9941 - val_

   1112/Unknown - 89s 80ms/step

InvalidArgumentError: Graph execution error:

Detected at node model/conv2d/Conv2D-0-TransposeNHWCToNCHW-LayoutOptimizer defined at (most recent call last):
<stack traces unavailable>
transpose expects a vector of size 3. But input(1) is a vector of size 4
	 [[{{node model/conv2d/Conv2D-0-TransposeNHWCToNCHW-LayoutOptimizer}}]] [Op:__inference_test_function_4323771]